In [ ]:
#calculates the number of unique medications prescribed over the 120 days prior to care home admission (exlcuding non-drug chapters of the BNF group)
CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_polypharmacy` as (
with a as (
  SELECT person_id, first_episodestartdate
  from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`),

b as (
SELECT person_id,idmultilexdmd,idmultilexproduct,datemedicationstart, FROM `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srprimarycaremedication` where person_id in(select person_id from a) order by idmultilexproduct),

c as (
  select
  a.person_id, 
  cast(a.first_episodestartdate as datetime) ch_entrydate,
  b.datemedicationstart,
  b.idmultilexproduct,
  b.idmultilexdmd
from a
LEFT JOIN b on
a.person_id = b.person_id),

d as (
select
*
from c 
where datemedicationstart between datetime_sub(ch_entrydate,INTERVAL 120 day) AND c.ch_entrydate),

e as (select BNF_Code,BNF_Name, cast(SNOMED_CODE as string) as SNOMEDCODE_STRING
  from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_BNF_DMD_SNOMED_lkp`),


f as (select 
d.*,
e.BNF_Code,
e.BNF_Name
from d 
left join e
on d.idmultilexdmd = e.SNOMEDCODE_STRING),

g as (
select 
count(*) as no_prescriptions,
SUBSTRING(BNF_code,1,6) as BNF_sub_sub_chapter,
person_id,
BNF_code,
BNF_name,
from f
where BNF_Code not like '2%' AND BNF_Code not like '19%'
group by person_id, BNF_code, BNF_name
order by person_id),


h as (
select
count(*) no_prescriptions,
person_id,
BNF_sub_sub_chapter
FROM g 
group by person_id, g.BNF_sub_sub_chapter),

i as (
select 
person_id,
count(*) as no_medication,
from h 
group by person_id)

SELECT 
person_id,
case when no_medication > 4 then '5+'
when no_medication >9 then '10+' end as polypharmacy_status,
no_medication
from i)
